Explorando dados

In [38]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

navegador = webdriver.Chrome()
navegador.get('https://www.iso.org/standards.html')

search_box = navegador.find_element(By.XPATH, '//*[@id="siteSearch"]/div/input')
search_box.send_keys('ISO')
search_box.send_keys(Keys.RETURN)
time.sleep(10)

procura os titulos e salva

In [42]:
def acha_iso():
    titulos = navegador.find_elements(By.CLASS_NAME, 'media-body')
    Iso = []
    for titulo in titulos:
        h5 = titulo.find_element(By.CLASS_NAME, 'h5')
        firstP = h5.find_element(By.TAG_NAME, 'a')
        nome = firstP.get_attribute('title')
        Iso.append(nome)
    return Iso

def prox_pagina():
    Iso = []
    paginas = navegador.find_elements(By.CLASS_NAME, 'pagination')
    for pagina in paginas:
        Iso += acha_iso()
        page = pagina.find_element(By.CLASS_NAME, 'page-item ')
        link = page.find_element(By.CLASS_NAME, 'page-link')
        tag = link.find_element(By.CLASS_NAME, 'bi-chevron-right')
        WebDriverWait(navegador, 10).until(EC.visibility_of(tag))
        navegador.execute_script("arguments[0].click();", tag)
    return Iso


iso = prox_pagina()
print(iso)

['ISO 4197:1989 Equipment for working the soil — Hoe blades — Fixing dimensions', 'ISO 4186:1980 Asparagus — Guide to storage', 'ISO 4178:1980 Complete, filled transport packages — Distribution trials — Information to be recorded', 'ISO 4132:1979 Unplasticized polyvinyl chloride (PVC) and metal adaptor fittings for pipes under pressure — Laying lengths and size of threads — Metric series', 'ISO 4072:1982 Green coffee in bags — Sampling', 'ISO 4038:1996 Road vehicles — Hydraulic braking systems — Simple flare pipes, tapped holes, male fittings and hose end fittings', 'ISO 4021:1992 Hydraulic fluid power — Particulate contamination analysis — Extraction of fluid samples from lines of an operating system', 'ISO 3974:1977 Definitions of living animals for slaughter — Ovines', 'ISO 3970:1977 Modular units for machine tool construction — Integral way columns — Floor-mounted type', 'ISO 3965:1990 Agricultural wheeled tractors — Maximum speeds — Method of determination']
